In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from hidmed import *
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.lines import Line2D

## Evaluate estimators

In [ ]:
setups = ["a", "b", "c"]
sample_sizes = [600, 3000, 6000]
dims = [1, 5]
num_evals = 100

results = {}

for setup in setups:
    for n in sample_sizes:
        for dim in dims:
            if dim == 1:
                xdim, zdim, wdim, mdim, udim = 1, 1, 1, 1, 1
            else:
                xdim, zdim, wdim, mdim, udim = 5, 2, 2, 2, 1

            # set up model
            datagen = LinearHidMedDGP(xdim, zdim, wdim, mdim, udim, setup=setup, seed=0)
            true_psi = datagen.true_psi()

            # estimation
            for estimator in [
                ProximalMultiplyRobust,
                ProximalInverseProbWeighting,
                ProximalOutcomeRegression,
            ]:
                print(
                    f"Running {estimator.__name__} for {dim}-dimensional case, n={n}, setup={setup}"
                )

                # hyperparameter tuning
                predictor = estimator(setup, folds=2)
                dataset = datagen.sample_dataset(n, seed=1)
                predictor.fit(dataset)

                # evaluation
                res = {
                    "estimate": np.zeros(num_evals),
                    "bias": np.zeros(num_evals),
                    "mse": np.zeros(num_evals),
                    "anb": np.zeros(num_evals),
                    "covered": np.zeros(num_evals),
                    "ci_width": np.zeros(num_evals),
                }
                for i in tqdm(range(1, num_evals + 1)):
                    if i == 1:
                        eval_data = None
                    else:
                        eval_data = datagen.sample_dataset(2 * n // 3, seed=i)
                    estimate = predictor.evaluate(
                        eval_data=eval_data, reduce=False, verbose=False
                    )

                    res["estimate"][i - 1] = np.mean(estimate)
                    res["bias"][i - 1] = np.mean(estimate) - true_psi
                    res["mse"][i - 1] = calculate_mse(estimate, true_psi)
                    res["anb"][i - 1] = absolute_normalized_bias(estimate, true_psi)
                    res["covered"][i - 1] = is_covered(estimate, true_psi)
                    res["ci_width"][i - 1] = confidence_interval(estimate)

                results[dim, setup, n, estimator.__name__] = res
                print("bias", np.mean(res["bias"]))
                print("mse", np.mean(res["mse"]))
                print("coverage", np.mean(res["covered"]))

pickle.dump(results, open("assets/results.pkl", "wb"))

Running ProximalMultiplyRobust for 1-dimensional case, n=600, setup=a
==== Fitting fold 1 (200 data points)
Bridge q params: {'lambda1': 0.028853998118144278, 'lambda2': 0.07607792007016918, 'gamma1': 0.1, 'gamma2': 0.1}, score: 0.6745141312270466
Bridge h params: {'lambda1': 0.046852450976837844, 'lambda2': 0.028853998118144278, 'gamma1': 0.1, 'gamma2': 0.1}, score: 0.0031144122867382104
eta params: {'alpha': 1.0000000000000004e-06, 'gamma': 0.1776796228004489}, r2: 0.9662709929019072
==== Fitting fold 2 (200 data points)
Bridge q params: {'lambda1': 0.028853998118144278, 'lambda2': 0.046852450976837844, 'gamma1': 0.1, 'gamma2': 0.1}, score: 0.6191123431267872
Bridge h params: {'lambda1': 0.12353355697579951, 'lambda2': 0.028853998118144278, 'gamma1': 0.1, 'gamma2': 0.1}, score: 0.0063812411744876285
eta params: {'alpha': 5.414704210271895e-06, 'gamma': 0.025141601599284524}, r2: 0.9575210520374687


100%|██████████| 100/100 [00:01<00:00, 60.24it/s]


bias -0.1883379827271081
mse 0.03771137772773529
coverage 0.0
Running ProximalInverseProbWeighting for 1-dimensional case, n=600, setup=a
==== Fitting fold 1 (200 data points)
Bridge q params: {'lambda1': 0.028853998118144278, 'lambda2': 0.07607792007016918, 'gamma1': 0.1, 'gamma2': 0.1}, score: 0.6745141312270466
==== Fitting fold 2 (200 data points)
Bridge q params: {'lambda1': 0.028853998118144278, 'lambda2': 0.046852450976837844, 'gamma1': 0.1, 'gamma2': 0.1}, score: 0.6191123431267872


100%|██████████| 100/100 [00:00<00:00, 176.59it/s]


bias -0.3390497819259635
mse 0.11930703511137003
coverage 0.0
Running ProximalOutcomeRegression for 1-dimensional case, n=600, setup=a
==== Fitting fold 1 (200 data points)
Bridge h params: {'lambda1': 0.046852450976837844, 'lambda2': 0.028853998118144278, 'gamma1': 0.1, 'gamma2': 0.1}, score: 0.0031144122867382104
eta params: {'alpha': 1.024479799068589e-06, 'gamma': 0.1794743164812678}, r2: 0.9662714203188966
==== Fitting fold 2 (200 data points)
Bridge h params: {'lambda1': 0.12353355697579951, 'lambda2': 0.028853998118144278, 'gamma1': 0.1, 'gamma2': 0.1}, score: 0.0063812411744876285
eta params: {'alpha': 2.1223584044330337e-05, 'gamma': 0.033090983265046295}, r2: 0.9575270208676437


...

/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/hom

/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/users/yalan/.local/lib/python3.9/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/hom

Treatment prob params: {'C': 1000000.0, 'degree': 1} log_loss:  0.683
Bridge h params: {'lambda1': 0.04573050519273264, 'lambda2': 2.2101234137112025, 'gamma1': 0.1, 'gamma2': 0.1}, score: 0.005163719629570915
eta params: {'alpha': 0.0008287033297256857, 'gamma': 0.15802020991007215}, r2: 0.6814540283677042


100%|██████████| 100/100 [00:24<00:00,  4.11it/s]


bias 0.08527820225731655
mse 0.007279036773479407
coverage 0.0
Running ProximalMultiplyRobust for 5-dimensional case, n=6000, setup=b
==== Fitting fold 1 (2000 data points)
Treatment prob params: {'C': 0.1, 'degree': 1} log_loss:  0.691


## Visualize results

In [ ]:
results = pickle.load(open("assets/results.pkl", "rb"))

In [ ]:
dim = 1
estimators = [
    ProximalMultiplyRobust,
    ProximalInverseProbWeighting,
    ProximalOutcomeRegression,
]
for setup in setups:
    idx = 1
    positions = []
    for col in range(len(sample_sizes)):
        for i in range(len(estimators)):
            positions.append(idx)
            idx += 1
        idx += 2

    labels = [
        sample_sizes[col] if i == 2 else " "
        for col in range(len(sample_sizes))
        for i in range(3)
    ]

    fig, ax = plt.subplots(figsize=(3 * 2, 4))
    plotdata = []
    for n in sample_sizes:
        for estimator in [est.__name__ for est in estimators]:
            plotdata.append(np.array(results[dim, setup, n, estimator]["bias"]))

    bplot = ax.boxplot(
        np.vstack(plotdata).T,
        positions=positions,
        labels=labels,
        patch_artist=True,
        widths=1,
        showmeans=True,
    )

    base_colors = ["cornflowerblue", "mediumaquamarine", "khaki"]
    colors = [c for _ in range(len(sample_sizes)) for c in base_colors]
    for patch, color in zip(bplot["boxes"], colors):
        patch.set_facecolor(color)

    custom_lines = [Line2D([0], [0], color=c, lw=4) for c in base_colors]
    ax.legend(custom_lines, ["POR", "PIPW", "PMR"])

    ax.axhline(y=0.0, color="k", linestyle="--", zorder=3)
    # plt.ylim(-2.5, 2.5)
    plt.xticks(positions[1 :: len(estimators)], sample_sizes)
    plt.xlabel("Sample Size")
    plt.ylabel("Bias")
    plt.tight_layout()
    # plt.savefig("assets/prox_hidmed_1d_sample_sizes_setup_%s.png" % mode, dpi=500)
    plt.show()

In [ ]:
for dim in dims:
    for setup in setups:
        for n in sample_sizes:
            # Generate the key for the current combination
            key = (dim, setup, n, "ProximalMultiplyRobust")

            # Check if the key exists in the dictionary
            if key in results:
                # Retrieve the sub-dictionary for the current key
                data = results[key]

                # Print averages for each metric
                print(f"Summary for {key}:")
                for metric in ["estimate", "bias", "mse", "anb", "covered", "ci_width"]:
                    average = np.mean(data[metric])
                    print(f"  Average {metric}: {average:.4f}")
                print()